In [1]:
#pip install joblib
#ee.Initialize()

import multiprocessing
num_cores = multiprocessing.cpu_count()
print(f"Number of CPU cores: {num_cores}")



Number of CPU cores: 24


In [2]:
import geopandas as gpd
import ee
import pandas as pd
from joblib import Parallel, delayed

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
# Authenticate to Earth Engine
#ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AVHEtk5f9hGx9aksCTq5O_pQ8H4-in4Z_-arrFTwMqq7_cCu2QH2ilBAHPQ



Successfully saved authorization token.


In [3]:
#pip install earthengine-api --upgrade



filepath = "C:\\Users\\admin-magstadt\\Desktop\\FireOccurence_3_27_2023\\National_USFS_Fire_Occurrence_Point_R123456_noNull_post1981.shp"


In [ ]:
#data = gpd.read_file(filepath)#, rows=slice(0, 1))


In [ ]:


# import time

# start_time = time.time()

# # Load the ERA5-Land Image Collection
# era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

# # Read in the shapefile
# data = gpd.read_file(filepath, rows=slice(0, 4))

# def extract_temp(row):
#     # Get the latitude and longitude of the point
#     lon, lat = row.geometry.coords[0]

#     # Get the date of interest
#     date = ee.Date(row.DISCOVERYD)

#     # Create a point geometry
#     point = ee.Geometry.Point([lon, lat])

#     # Filter the ERA5-Land Image Collection to the date of interest and the point
#     image = era5.filterBounds(point).filterDate(date, date.advance(1, 'day')).mean()

#     # Define the list of variables to extract
#     var_list = ['temperature_2m', 'dewpoint_temperature_2m', 'skin_temperature', 'surface_pressure', 'total_precipitation', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
#                 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_evaporation', 'potential_evaporation', 'evaporation_from_vegetation_transpiration', 'evaporation_from_the_top_of_canopy',
#                 'evaporation_from_bare_soil', 'surface_thermal_radiation_downwards', 'surface_net_thermal_radiation', 'surface_net_solar_radiation', 'surface_latent_heat_flux',
#                 'forecast_albedo', 'volumetric_soil_water_layer_1', 'soil_temperature_level_1']

#     # Extract the required variables from the image at the point location
#     values = image.select(var_list).reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132)

#     # Get the values for each variable and return as a tuple
#     return tuple(values.get(var).getInfo() for var in var_list)


# # Apply the extract_temp function to each row in the DataFrame to add new columns with temperature and dewpoint temperature data
# data[['temperature_2m', 'dewpoint_temperature_2m', 'skin_temperature', 'surface_pressure', 'total_precipitation', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
#       'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_evaporation', 'potential_evaporation', 'evaporation_from_vegetation_transpiration', 'evaporation_from_the_top_of_canopy',
#       'evaporation_from_bare_soil', 'surface_thermal_radiation_downwards', 'surface_net_thermal_radiation', 'surface_net_solar_radiation', 'surface_latent_heat_flux',
#       'forecast_albedo', 'volumetric_soil_water_layer_1', 'soil_temperature_level_1']] = data.apply(lambda row: pd.Series(extract_temp(row)), axis=1)

# end_time = time.time()
# total_time = end_time - start_time
# print("Total time:", total_time)


# Parallel - but nonbatch

In [ ]:
import time
from joblib import Parallel, delayed

start_time = time.time()

# Load the ERA5-Land Image Collection
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

# Read in the shapefile
data = gpd.read_file(filepath, rows=slice(0, 1000))

def extract_temp(row):
    ee.Initialize()
    # Get the latitude and longitude of the point
    lon, lat = row.geometry.coords[0]

    # Get the date of interest
    date = ee.Date(row.DISCOVERYD)

    # Create a point geometry
    point = ee.Geometry.Point([lon, lat])

    # Filter the ERA5-Land Image Collection to the date of interest and the point
    #image = era5.filterBounds(point).filterDate(date, date.advance(1, 'day')).mean()
    
    start_time = ee.Date(date).update(hour=14)
    print("d")
    end_time = ee.Date(date).update(hour=15)
    print(end_time)

    # Filter the ERA5-Land Image Collection to the date and time range of interest and the point
    image = era5.filterBounds(point).filterDate(start_time, end_time).mean()

    # Define the list of variables to extract
    var_list = ['temperature_2m', 'dewpoint_temperature_2m', 'skin_temperature', 'surface_pressure', 'total_precipitation', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
                'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_evaporation', 'potential_evaporation', 'evaporation_from_vegetation_transpiration', 'evaporation_from_the_top_of_canopy',
                'evaporation_from_bare_soil', 'surface_thermal_radiation_downwards', 'surface_net_thermal_radiation', 'surface_net_solar_radiation', 'surface_latent_heat_flux',
                'forecast_albedo', 'volumetric_soil_water_layer_1', 'soil_temperature_level_1']

    # Extract the required variables from the image at the point location
    values = image.select(var_list).reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132)

    # Get the values for each variable and return as a tuple
    return tuple(values.get(var).getInfo() for var in var_list)

# Define the number of parallel jobs
n_jobs = 22

# Apply the extract_temp function to each row in the DataFrame in parallel
results = Parallel(n_jobs=n_jobs)(delayed(extract_temp)(row) for i, row in data.iterrows())

# Add the extracted data to the DataFrame
data[['temperature_2m', 'dewpoint_temperature_2m', 'skin_temperature', 'surface_pressure', 'total_precipitation', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
      'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_evaporation', 'potential_evaporation', 'evaporation_from_vegetation_transpiration', 'evaporation_from_the_top_of_canopy',
      'evaporation_from_bare_soil', 'surface_thermal_radiation_downwards', 'surface_net_thermal_radiation', 'surface_net_solar_radiation', 'surface_latent_heat_flux',
      'forecast_albedo', 'volumetric_soil_water_layer_1', 'soil_temperature_level_1']] = pd.DataFrame(results)

end_time = time.time()
total_time = end_time - start_time
print("Total time:", total_time)


# Parallel - batchcsv 

In [4]:
#data = gpd.read_file(filepath)
#len(data)

386188

In [10]:
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

In [22]:
import time
from joblib import Parallel, delayed

start_time = time.time()

# Load the ERA5-Land Image Collection
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

# Read in the shapefile
#data = gpd.read_file(filepath)

# Define the number of parallel jobs
n_jobs = 16

# Define the number of rows per subset
batch_size = 500

# Loop through the subsets of the data
for i in range(340000, len(data), batch_size):
    # Get the subset of rows
    #subset = data.loc[i:i+batch_size]
    #subset = data.iloc[i:i+batch_size]
    #data.loc[i:i+batch_size] = subset
    subset = gpd.read_file(filepath, rows=slice(i, i+batch_size))
    
    print(i)
    #ee.Initialize()

    def extract_temp(row):
        ee.Initialize(project='ee-magstadt')
        #ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
        # Load the ERA5-Land Image Collection
        #era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
        
        # Get the latitude and longitude of the point
        lon, lat = row.geometry.coords[0]

        # Get the date of interest
        date = ee.Date(row.DISCOVERYD)

        # Create a point geometry
        point = ee.Geometry.Point([lon, lat])

        # Filter the ERA5-Land Image Collection to the date and time range of interest and the point
        start_time = ee.Date(date).update(hour=1)
        end_time = ee.Date(date).update(hour=23)
        image = era5.filterBounds(point).filterDate(start_time, end_time).max()

        # Define the list of variables to extract
        var_list = ['temperature_2m', 'dewpoint_temperature_2m', 'skin_temperature', 'surface_pressure', 'total_precipitation', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
                    'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_evaporation', 'potential_evaporation', 'evaporation_from_vegetation_transpiration', 'evaporation_from_the_top_of_canopy',
                    'evaporation_from_bare_soil', 'surface_thermal_radiation_downwards', 'surface_net_thermal_radiation', 'surface_net_solar_radiation', 'surface_latent_heat_flux',
                    'forecast_albedo', 'volumetric_soil_water_layer_1', 'soil_temperature_level_1']

        # Extract the required variables from the image at the point location
        values = image.select(var_list).reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132)
        
        #ee.reset()

        # Get the values for each variable and return as a tuple
        return tuple(values.get(var).getInfo() for var in var_list)
        
        
        # row_values = []
        # for var in var_list:
        #     value = values.get(var).getInfo()
        #     if value is None:
        #         print(f'No value for {var} in row {row.name}')
        #         row_values.append('')
        #     else:
        #         row_values.append(str(value))
        # return tuple(row_values)
        
        
        

    # Apply the extract_temp function to each row in the DataFrame in parallel
    results = Parallel(n_jobs=n_jobs)(delayed(extract_temp)(row) for _, row in subset.iterrows())

    # Add the extracted data to the DataFrame
    subset[['temperature_2m', 'dewpoint_temperature_2m', 'skin_temperature', 'surface_pressure', 'total_precipitation', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
          'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_evaporation', 'potential_evaporation', 'evaporation_from_vegetation_transpiration', 'evaporation_from_the_top_of_canopy',
          'evaporation_from_bare_soil', 'surface_thermal_radiation_downwards', 'surface_net_thermal_radiation', 'surface_net_solar_radiation', 'surface_latent_heat_flux',
          'forecast_albedo', 'volumetric_soil_water_layer_1', 'soil_temperature_level_1']] = pd.DataFrame(results)
    
    filename = f"C:\\Users\\admin-magstadt\\Desktop\\FireOccurence_3_27_2023\\OutputData\\FireOccurenceSamples_{i}_{i+batch_size}.csv"
    # Export the GeoDataFrame to CSV
    subset.to_csv(filename, index=False)
    
    #pq.write_table(subset, filename)
    #gpd.to_parquet(subset, filename)


340000
340500
341000
341500
342000
342500
343000
343500
344000
344500
345000
345500
346000
346500
347000
347500
348000
348500
349000
349500
350000
350500
351000
351500
352000
352500
353000
353500
354000
354500
355000
355500
356000
356500
357000
357500
358000
358500
359000
359500
360000
360500
361000
361500
362000
362500
363000
363500
364000
364500
365000
365500
366000
366500
367000
367500
368000
368500
369000
369500
370000
370500
371000
371500
372000
372500
373000
373500
374000
374500
375000
375500
376000
376500
377000
377500
378000
378500
379000
379500
380000
380500
381000
381500
382000
382500
383000
383500
384000
384500
385000
385500
386000


In [ ]:
type(data)

In [ ]:
# import geopandas as gpd
# import ee
# import pandas as pd
# # Authenticate to Earth Engine
# #ee.Authenticate()
# ee.Initialize()

# import time

# start_time = time.time()


# # Load the ERA5-Land Image Collection
# era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

# # Read in the shapefile
# data = gpd.read_file(filepath, rows=slice(0, 10))

# def extract_temp(row):
#     # Get the latitude and longitude of the point
#     lon, lat = row.geometry.coords[0]
    
#     # Get the date of interest
#     date = ee.Date(row.DISCOVERYD)
    
#     # Create a point geometry
#     point = ee.Geometry.Point([lon, lat])
    
#     # Filter the ERA5-Land Image Collection to the date of interest and the point
#     image = era5.filterBounds(point).filterDate(date, date.advance(1, 'day')).mean()
    
#     # Extract the required variables from the image at the point location
#     temperature_2m = image.select('temperature_2m').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('temperature_2m')
#     print("d")
#     dewpoint_temperature_2m = image.select('dewpoint_temperature_2m').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('dewpoint_temperature_2m')
#     print("d")

#     skin_temperature = image.select('skin_temperature').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('skin_temperature')
#     surface_pressure = image.select('surface_pressure').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('surface_pressure')
#     total_precipitation = image.select('total_precipitation').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('total_precipitation')
#     leaf_area_index_high_vegetation = image.select('leaf_area_index_high_vegetation').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('leaf_area_index_high_vegetation')
#     leaf_area_index_low_vegetation = image.select('leaf_area_index_low_vegetation').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('leaf_area_index_low_vegetation')
    
#     u_component_of_wind_10m = image.select('u_component_of_wind_10m').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('u_component_of_wind_10m')
#     v_component_of_wind_10m = image.select('v_component_of_wind_10m').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('v_component_of_wind_10m')
#     total_evaporation = image.select('total_evaporation').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('total_evaporation')
#     potential_evaporation = image.select('potential_evaporation').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('potential_evaporation')
#     evaporation_from_vegetation_transpiration = image.select('evaporation_from_vegetation_transpiration').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('evaporation_from_vegetation_transpiration')
#     evaporation_from_the_top_of_canopy = image.select('evaporation_from_the_top_of_canopy').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('evaporation_from_the_top_of_canopy')
    
#     evaporation_from_bare_soil = image.select('evaporation_from_bare_soil').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('evaporation_from_bare_soil')
#     surface_thermal_radiation_downwards = image.select('surface_thermal_radiation_downwards').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('surface_thermal_radiation_downwards')
#     surface_net_thermal_radiation = image.select('surface_net_thermal_radiation').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('surface_net_thermal_radiation')
#     surface_net_solar_radiation = image.select('surface_net_solar_radiation').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('surface_net_solar_radiation')
#     surface_latent_heat_flux = image.select('surface_latent_heat_flux').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('surface_latent_heat_flux')
#     forecast_albedo = image.select('forecast_albedo').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('forecast_albedo')
#     volumetric_soil_water_layer_1 = image.select('volumetric_soil_water_layer_1').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('volumetric_soil_water_layer_1')
#     soil_temperature_level_1 = image.select('soil_temperature_level_1').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('soil_temperature_level_1')
    
#     return (
#     temperature_2m.getInfo(),
#     dewpoint_temperature_2m.getInfo(),
#     skin_temperature.getInfo(),
#     surface_pressure.getInfo(),
#     total_precipitation.getInfo(),
#     leaf_area_index_high_vegetation.getInfo(),
#     leaf_area_index_low_vegetation.getInfo(),
#     u_component_of_wind_10m.getInfo(),
#     v_component_of_wind_10m.getInfo(),
#     total_evaporation.getInfo(),
#     potential_evaporation.getInfo(),
#     evaporation_from_vegetation_transpiration.getInfo(),
#     evaporation_from_the_top_of_canopy.getInfo(),
#     evaporation_from_bare_soil.getInfo(),
#     surface_thermal_radiation_downwards.getInfo(),
#     surface_net_thermal_radiation.getInfo(),
#     surface_net_solar_radiation.getInfo(),
#     surface_latent_heat_flux.getInfo(),
#     forecast_albedo.getInfo(),
#     volumetric_soil_water_layer_1.getInfo(),
#     soil_temperature_level_1.getInfo()
# )
                                                                           
# # Apply the extract_temp function to each row in the DataFrame to add new columns with temperature and dewpoint temperature data
# data[['temperature_2m', 'dewpoint_temperature_2m','skin_temperature','surface_pressure','total_precipitation','leaf_area_index_high_vegetation','leaf_area_index_low_vegetation',
#       'u_component_of_wind_10m','v_component_of_wind_10m','total_evaporation','potential_evaporation','evaporation_from_vegetation_transpiration','evaporation_from_the_top_of_canopy',
#       'evaporation_from_bare_soil','surface_thermal_radiation_downwards','surface_net_thermal_radiation','surface_net_solar_radiation','surface_latent_heat_flux',
#       'forecast_albedo','volumetric_soil_water_layer_1','soil_temperature_level_1']] = data.apply(lambda row: pd.Series(extract_temp(row)), axis=1)
# #data                                                                            

# end_time = time.time()
# total_time = end_time - start_time
# print("Total time:", total_time)


In [ ]:
# import geopandas as gpd
# import ee
# import pandas as pd
# # Authenticate to Earth Engine
# #ee.Authenticate()
# ee.Initialize()

# # Load the ERA5-Land Image Collection
# era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

# # Read in the shapefile
# data = gpd.read_file(filepath, rows=slice(0, 5))
# date = data.DISCOVERYD

# # Define a function to extract temperature and dewpoint temperature data for a single point and date
# def extract_temp(row):
#     # Get the latitude and longitude of the point
#     lon, lat = row.geometry.coords[0]
    
#     # Get the date of interest
#     date = ee.Date(row.DISCOVERYD)
    
#     # Create a point geometry and a buffer around it
#     point = ee.Geometry.Point([lon, lat])
#     #buffer = point.buffer(10)  # 1000 km buffer around the point (1 degree = 111.32 km)
    
#     # Filter the ERA5-Land Image Collection to the date of interest and the buffer around the point
#     #image = era5.filterBounds(buffer).filterDate(date, date.advance(1, 'day')).mean()
#     image = era5.filterBounds(point).filterDate(date, date.advance(1, 'day')).mean()

#     # Extract the 2m temperature and dewpoint temperature from the image at the point location
#     temp = image.select('temperature_2m').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('temperature_2m')
#     dew_temp = image.select('dewpoint_temperature_2m').reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=11132).get('dewpoint_temperature_2m')
    
#     # Return the temperature and dewpoint temperature values
#     return temp.getInfo(), dew_temp.getInfo()

# # Apply the extract_temp function to each row in the DataFrame to add new columns with temperature and dewpoint temperature data
# data[['temperature_2m', 'dewpoint_temperature_2m']] = data.apply(lambda row: pd.Series(extract_temp(row)), axis=1)
# data


In [ ]:
"C:\\Users\\admin-magstadt\\Desktop\\FireOccurence_3_27_2023\\OutputData"